**Set environment**

In [37]:
source ../config_duke.sh -v

**Input**

In [2]:
ls -d ${FD_RES}/data/*/

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_DMSO/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_Dex/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_DMSO/
/gpfs/fs1/data/reddylab/Kuei/out/proj_com

In [3]:
ls ${FD_RES}/data/Input1/

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed  target_PER1.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


In [4]:
ls ${FD_RES}/data/Input1_20x/

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed  target_PER1.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


## Count fragments for each chromosome

**Test Loop**

In [5]:
### Loop chromosomes
CHROMS=($(seq 1 22) X Y)
for idx in {0..23}; do
    echo "Index: $idx; Chromosome ${CHROMS[$idx]}"
done

Index: 0; Chromosome 1
Index: 1; Chromosome 2
Index: 2; Chromosome 3
Index: 3; Chromosome 4
Index: 4; Chromosome 5
Index: 5; Chromosome 6
Index: 6; Chromosome 7
Index: 7; Chromosome 8
Index: 8; Chromosome 9
Index: 9; Chromosome 10
Index: 10; Chromosome 11
Index: 11; Chromosome 12
Index: 12; Chromosome 13
Index: 13; Chromosome 14
Index: 14; Chromosome 15
Index: 15; Chromosome 16
Index: 16; Chromosome 17
Index: 17; Chromosome 18
Index: 18; Chromosome 19
Index: 19; Chromosome 20
Index: 20; Chromosome 21
Index: 21; Chromosome 22
Index: 22; Chromosome X
Index: 23; Chromosome Y


In [6]:
### Loop folders
FD_BEDS=($(ls -d ${FD_RES}/data/Input?_20x/))
for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    echo "SAMPLE: ${SAMPLE}; FD_BED: ${FD_BED}"
done

SAMPLE: Input1_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
SAMPLE: Input2_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
SAMPLE: Input3_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
SAMPLE: Input4_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
SAMPLE: Input5_20x; FD_BED: /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/


**RUN**

In [7]:
echo ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log
all


In [74]:
source ../config_duke.sh

sbatch -p ${NODE} \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/count_fragment_chrom_input_20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=chr${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/data/Input?_20x/))
FD_OUT=${FD_RES}/count_fragment
FN_BED=${TARGET}.bed

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_CNT=${FD_OUT}/${SAMPLE}
    FN_CNT=${TARGET}.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_CNT}/${FN_CNT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_CNT}
    
    ### count the fragments and arrange output to a proper bed file
    cat ${FD_BED}/${FN_BED} |\
        sort    |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        sort -k2,2n |\
        gzip -c > ${FD_CNT}/${FN_CNT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_CNT}/${FN_CNT}
    zcat ${FD_CNT}/${FN_CNT} | head -n 3
    echo
done

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 26208747


**CHECK**

In [76]:
tail ${FD_LOG}/count_fragment_chrom_input_20x.*.txt

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/count_fragment_chrom_input_20x.0.txt <==
chr1	13627	14515

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/chr1.bed.gz
chr1	12225	13098	1
chr1	13627	14515	2
chr1	13868	14793	1

Done!
Run Time: 35 minutes and 33 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/count_fragment_chrom_input_20x.10.txt <==
chr11	102297	103363

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/chr11.bed.gz
chr11	72984	73929	1
chr11	102190	103214	1
chr11	102297	103363	2

Done!
Run Time: 21 minutes and 27 seconds

==> /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/count_fragment_chrom_input_20x.11.txt <==
chr12	30619	31502

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input5_20x/chr12.bed.gz
chr12	28510	29379	1
chr12	28535	29429	1
ch

In [79]:
head -20 ${FD_LOG}/count_fragment_chrom_input_20x.21.txt

Slurm Array Index:  21
Time Stamp:         10-01-21+17:04:30

RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr22.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/chr22.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr22.bed
chr22	10519569	10520481
chr22	10519675	10520727
chr22	10524043	10525223

Show the first few lines of the output file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/count_fragment/Input1_20x/chr22.bed.gz
chr22	10519569	10520481	1
chr22	10519675	10520727	1
chr22	10524043	10525223	1



In [41]:
source ../config_duke.sh

sbatch -p scavenger \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/count_fragment_chrom.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=chr${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_WRK}/data/*/))
FD_OUT=${FD_WRK}/count_fragment
FN_BED=${TARGET}.bed

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_CNT=${FD_OUT}/${SAMPLE}
    FN_CNT=${TARGET}.bed
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_CNT}/${FN_CNT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_CNT}
    
    ### count the fragments and arrange output to a proper bed file
    cat ${FD_BED}/${FN_BED} |\
        sort    |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        sort -k2,2n > ${FD_CNT}/${FN_CNT}
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_CNT}/${FN_CNT}
    head -n 3 ${FD_CNT}/${FN_CNT}
    echo
done
echo "Done!"

EOF

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Submitted batch job 14789207


**CHECK**

In [43]:
ls ${FD_LOG}/count_fragment_chrom.17.txt

/work/kk319/out/CombEffect_STARR/log/count_fragment_chrom.17.txt


In [44]:
cat ${FD_LOG}/count_fragment_chrom.17.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  17
RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
Output file:  /work/kk319/out/CombEffect_STARR/count_fragment/Input/chr18.bed

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
chr18	21483	22446
chr18	21483	22446
chr18	28214	29137

Show the first few lines of the output file
/work/kk319/out/CombEffect_STARR/count_fragment/Input/chr18.bed
chr18	15221	16301	1
chr18	20556	21638	1
chr18	20605	21575	1

++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input1//chr18.bed
Output f